In [94]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from itertools import product

In [11]:
smiles = "CC[C@H]1C(=O)N(CC(=O)N([C@H](C(=O)N[C@H](C(=O)N([C@H](C(=O)N[C@H](C(=O)N[C@@H](C(=O)N([C@H](C(=O)N([C@H](C(=O)N[C@H](C(=O)N([C@H](C(=O)N1)[C@@H]([C@H](C)C/C=C/C)O)C)C(C)C)CC(C)C)C)CC(C)C)C)C)C)CC(C)C)C)C(C)C)CC(C)C)C)C"
ref_pdb_path = "./CsE_atomname.pdb"

In [12]:
smiles_mol = Chem.MolFromSmiles(smiles)
ref_mol = Chem.MolFromPDBFile(ref_pdb_path)
# mol = Chem.MolFromPDBFile(pdb_path)

ref_mol = AllChem.AssignBondOrdersFromTemplate(smiles_mol,ref_mol)
# mol = AllChem.AssignBondOrdersFromTemplate(smiles_mol, mol)
# order = list(mol.GetSubstructMatches(ref_mol)[0])
# mol = Chem.RenumberAtoms(mol, order)


(193, 193)

In [27]:
print(Chem.MolToPDBBlock(mol))

ATOM      1  N   MEB A   1       9.764  -2.140  -3.602  1.00  0.00           N  
ATOM      2  CN  MEB A   1       8.354  -2.296  -3.218  1.00  0.00           C  
ATOM      3  CA  MEB A   1      10.768  -3.022  -2.988  1.00  0.00           C  
ATOM      4  CB  MEB A   1      10.602  -3.439  -1.525  1.00  0.00           C  
ATOM      5  OG1 MEB A   1       9.421  -4.191  -1.237  1.00  0.00           O  
ATOM      6  CG2 MEB A   1      11.119  -2.408  -0.520  1.00  0.00           C  
ATOM      7  CD1 MEB A   1      12.577  -2.029  -0.786  1.00  0.00           C  
ATOM      8  CD2 MEB A   1      10.876  -2.826   0.932  1.00  0.00           C  
ATOM      9  CE  MEB A   1       9.532  -2.504   1.587  1.00  0.00           C  
ATOM     10  CZ  MEB A   1       8.961  -3.492   2.270  1.00  0.00           C  
ATOM     11  CH  MEB A   1       7.709  -3.116   3.065  1.00  0.00           C  
ATOM     12  C   MEB A   1      10.915  -4.247  -3.892  1.00  0.00           C  
ATOM     13  O   MEB A   1  

In [22]:
mol = Chem.AddHs(ref_mol)

In [55]:
def assign_hydrogen_pdbinfo(mol):
    """
    assumes all heavy atoms has complete PDB information
    """
    for idx,atm in enumerate(mol.GetAtoms()):
        if atm.GetPDBResidueInfo() is None and atm.GetAtomicNum() == 1:
            assert len(atm.GetNeighbors()) == 1, "String hydrogen with hypervalence at index {}".format(idx)
            heavy_atm = atm.GetNeighbors()[0]
            
            mi = Chem.AtomPDBResidueInfo()
            mi.SetName("H{}".format(heavy_atm.GetPDBResidueInfo().GetName().strip()))
            mi.SetResidueNumber(heavy_atm.GetPDBResidueInfo().GetResidueNumber())
            mi.SetResidueName(heavy_atm.GetPDBResidueInfo().GetResidueName())
            mi.SetChainId(heavy_atm.GetPDBResidueInfo().GetChainId())
            atm.SetMonomerInfo(mi)
    return mol

In [56]:
mol = assign_hydrogen_pdbinfo(Chem.AddHs(ref_mol))
print(Chem.MolToPDBBlock(mol))

ATOM      1  N   MEB A   1       9.764  -2.140  -3.602  1.00  0.00           N  
ATOM      2  CN  MEB A   1       8.354  -2.296  -3.218  1.00  0.00           C  
ATOM      3  CA  MEB A   1      10.768  -3.022  -2.988  1.00  0.00           C  
ATOM      4  CB  MEB A   1      10.602  -3.439  -1.525  1.00  0.00           C  
ATOM      5  OG1 MEB A   1       9.421  -4.191  -1.237  1.00  0.00           O  
ATOM      6  CG2 MEB A   1      11.119  -2.408  -0.520  1.00  0.00           C  
ATOM      7  CD1 MEB A   1      12.577  -2.029  -0.786  1.00  0.00           C  
ATOM      8  CD2 MEB A   1      10.876  -2.826   0.932  1.00  0.00           C  
ATOM      9  CE  MEB A   1       9.532  -2.504   1.587  1.00  0.00           C  
ATOM     10  CZ  MEB A   1       8.961  -3.492   2.270  1.00  0.00           C  
ATOM     11  CH  MEB A   1       7.709  -3.116   3.065  1.00  0.00           C  
ATOM     12  C   MEB A   1      10.915  -4.247  -3.892  1.00  0.00           C  
ATOM     13  O   MEB A   1  

In [116]:
df = pd.read_csv("CsE_dmso_noebounds.csv", sep = "\s", comment = "#")

/home/swang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [117]:
out = []
for i in df["Residue_name_1"]:
    if i[1] != "C":
        out.append(i[0] + "C" + i[1:])
    else: out.append(i)
df["Residue_name_1"] = out

out = []
for i in df["Residue_name_2"]:
    if i[1] != "C":
        out.append(i[0] + "C" + i[1:])
    else: out.append(i)
df["Residue_name_2"] = out

In [118]:
df

,Index,Residue_index_1,Residue_name_1,Residue_index_2,Residue_name_2,Upper_bound_[nm]
0,1,1,HCA,7,HCA,0.529
1,2,1,HCA,7,HCN,0.398
2,3,7,HCA,2,HCN,0.550
3,4,2,HCN,5,HCN,0.441
4,5,2,HCN,5,HCA,0.575
5,6,2,HCN,6,HCA,0.530
6,7,11,HCA,7,HCN,0.519
7,8,11,HCN,8,HCN,0.572
8,9,11,HCN,7,HCN,0.521
9,10,1,HCA,6,HCB,0.550


In [119]:
bmat = AllChem.GetMoleculeBoundsMatrix(Chem.AddHs(ref_mol))
named_index = np.array(["{}-{}".format(atm.GetPDBResidueInfo().GetResidueNumber(),atm.GetPDBResidueInfo().GetName().strip()) 
                        for atm in mol.GetAtoms()])
named_index

array(['1-N', '1-CN', '1-CA', '1-CB', '1-OG1', '1-CG2', '1-CD1', '1-CD2',
       '1-CE', '1-CZ', '1-CH', '1-C', '1-O', '2-N', '2-CA', '2-CB',
       '2-CG', '2-C', '2-O', '3-N', '3-CN', '3-CA', '3-C', '3-O', '4-N',
       '4-CN', '4-CA', '4-CB', '4-CG', '4-CD1', '4-CD2', '4-C', '4-O',
       '5-N', '5-CA', '5-CB', '5-CG1', '5-CG2', '5-C', '5-O', '6-N',
       '6-CN', '6-CA', '6-CB', '6-CG', '6-CD1', '6-CD2', '6-C', '6-O',
       '7-N', '7-CA', '7-CB', '7-C', '7-O', '8-N', '8-CA', '8-CB', '8-C',
       '8-O', '9-N', '9-CN', '9-CA', '9-CB', '9-CG', '9-CD1', '9-CD2',
       '9-C', '9-O', '10-N', '10-CN', '10-CA', '10-CB', '10-CG', '10-CD1',
       '10-CD2', '10-C', '10-O', '11-N', '11-CA', '11-CB', '11-CG1',
       '11-CG2', '11-C', '11-O', '1-HCN', '1-HCN', '1-HCN', '1-HCA',
       '1-HCB', '1-HOG1', '1-HCG2', '1-HCD1', '1-HCD1', '1-HCD1',
       '1-HCD2', '1-HCD2', '1-HCE', '1-HCZ', '1-HCH', '1-HCH', '1-HCH',
       '2-HN', '2-HCA', '2-HCB', '2-HCB', '2-HCG', '2-HCG', '2-HCG',
       '3

In [120]:
for idx,val in df.iterrows():
    a = np.nonzero(named_index == "{}-{}".format(val["Residue_index_1"], val["Residue_name_1"]))[0]
    b = np.nonzero(named_index == "{}-{}".format(val["Residue_index_2"], val["Residue_name_2"]))[0]
    for item in product(a,b):
        bmat[min(item), max(item)] = val["Upper_bound_[nm]"] * 10


In [121]:
def check_bmat(bmat):
    for i in range(bmat.shape[0] - 1):
        for j in range(i + 1, bmat.shape[1]):
            if bmat[i,j] < bmat[j,i]:
                raise ValueError("Lower bound {}{}{} bigger than upper bound {}{}{}".format(i,j, bmat[i,j],
                                                                                           j,i,bmat[j,i]))
    print("Boundmatrix is fine")
check_bmat(bmat)

Boundmatrix is fine
